<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Mission with HE power train

The following Notebook shows the user an example of how to size an aircraft usinf the newly developed power train builder along with the mission from FAST-OAD_CS23.

In [ ]:
import os.path as pth
import logging
import shutil

import pandas as pd

import fastoad.api as oad
import fastga_he.api as oad_he

DATA_FOLDER_PATH = "data"

WORK_FOLDER_PATH = "workdir"

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "full_sizing.yml")
PT_FILE = pth.join(WORK_FOLDER_PATH, "simple_assembly.yml")
# The following PT file is not used for sizing just to try out the network function
PT_FILE_QUAD = pth.join(WORK_FOLDER_PATH, "quad_assembly.yml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "full_sizing.xml")

# For having log messages on screen
logging.basicConfig(level=logging.WARNING, format="%(levelname)-8s: %(message)s")

In [ ]:
# We copy all the useful file in the workdir

shutil.copy(pth.join(DATA_FOLDER_PATH, "full_sizing.yml"), CONFIGURATION_FILE)
shutil.copy(pth.join(DATA_FOLDER_PATH, "simple_assembly.yml"), PT_FILE)
shutil.copy(pth.join(DATA_FOLDER_PATH, "quad_assembly.yml"), PT_FILE_QUAD)

Before trying to launch the analysis, let's make sure that the powertrain described in the power train file corresponds to what is expected. To do so, FAST-OAD-GA-HE offers a tool for repreenting the power train as a network. Its use is displayed in the following cell.

In [ ]:
NETWORK_FILE = pth.join(WORK_FOLDER_PATH, "network.html")

oad_he.power_train_network_viewer(power_train_file_path=PT_FILE, network_file_path=NETWORK_FILE)

from IPython.display import IFrame

IFrame(src=NETWORK_FILE, width="100%", height="500px")

In [ ]:
NETWORK_FILE_QUAD = pth.join(WORK_FOLDER_PATH, "network_quad.html")

oad_he.power_train_network_viewer(
    power_train_file_path=PT_FILE_QUAD, network_file_path=NETWORK_FILE_QUAD
)

from IPython.display import IFrame

IFrame(src=NETWORK_FILE_QUAD, width="100%", height="500px")

Let's now move on to the generation of inputs.

In [ ]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

Now we run the problem. As the performances computation was made to stall when not enough progress is made or when it has already converged and can't do more, we will catch those warnings for a cleaner display.

In [ ]:
import warnings
import openmdao.api as om

warnings.filterwarnings("ignore", category=om.OpenMDAOWarning)

configurator = oad.FASTOADProblemConfigurator(CONFIGURATION_FILE)
problem = configurator.get_problem(read_inputs=True)
problem.setup()
problem.set_val("data:weight:aircraft:MTOW", units="kg", val=1000.0)
problem.run_model()
problem.write_outputs()

print(problem.get_val("data:weight:aircraft:MTOW", units="kg"))

We can now use the API to create graphs based on the data saved during mission computation

In [ ]:
MISSION_DATA_FILE = pth.join(WORK_FOLDER_PATH, "mission_data.csv")
PT_DATA_FILE = pth.join(WORK_FOLDER_PATH, "power_train_data.csv")

perfo_viewer = oad_he.PerformancesViewer(
    power_train_data_file_path=PT_DATA_FILE,
    mission_data_file_path=MISSION_DATA_FILE,
    plot_height=800,
)

# Uncomment next lines if you want raw data
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 200)
# print(perfo_viewer.data)

And also display a breakdown of the weight of the components of the power train.

In [ ]:
fig = oad_he.power_train_mass_breakdown(
    problem.output_file_path,
    power_train_file_path=PT_FILE,
)
fig.update_layout(height=800)

In [ ]:
oad.variable_viewer(problem.output_file_path)

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)

from IPython.display import IFrame

IFrame(src=N2_FILE, width="100%", height="500px")

Optimization of the propeller efficiency for cruise conditions (OpenMDAO style).

In [ ]:
import openmdao.api as om

from fastga_he.models.propulsion.components.propulsor.propeller import PerformancesPropeller

problem = om.Problem()
model = problem.model
model.add_subsystem(
    "ivc_rpm",
    om.IndepVarComp(
        "data:propulsion:he_power_train:propeller:propeller_1:rpm_mission", 2300, units="1/min"
    ),
    promotes=["*"],
)
model.add_subsystem(
    "propeller_perf",
    PerformancesPropeller(propeller_id="propeller_1", number_of_points=1),
    promotes=["*"],
)
problem.driver = om.ScipyOptimizeDriver()
problem.driver.options["optimizer"] = "differential_evolution"
problem.driver.options["maxiter"] = 1000
problem.driver.options["tol"] = 1e-4

problem.model.add_design_var(
    "data:propulsion:he_power_train:propeller:propeller_1:solidity", lower=0.1, upper=0.4
)
problem.model.add_design_var(
    "data:propulsion:he_power_train:propeller:propeller_1:activity_factor", lower=100, upper=300
)
problem.model.add_design_var(
    "data:propulsion:he_power_train:propeller:propeller_1:blade_twist",
    lower=15,
    upper=35,
    units="deg",
)
problem.model.add_design_var(
    "data:propulsion:he_power_train:propeller:propeller_1:diameter",
    lower=40,
    upper=80,
    units="inch",
)
problem.model.add_objective("efficiency", scaler=-1.0)

problem.nonlinear_solver = om.NewtonSolver(solve_subsystems=True)
problem.nonlinear_solver.options["iprint"] = 0
problem.nonlinear_solver.options["maxiter"] = 100
problem.nonlinear_solver.options["rtol"] = 1e-4
problem.linear_solver = om.DirectSolver()

problem.model.approx_totals()

problem.setup()

problem.set_val("altitude", val=3000, units="ft")
problem.set_val("thrust", val=560.0, units="N")
problem.set_val("true_airspeed", val=90.0, units="knot")
problem.set_val("data:propulsion:he_power_train:propeller:propeller_1:solidity", 0.4)
problem.set_val("data:propulsion:he_power_train:propeller:propeller_1:activity_factor", 300)
problem.set_val(
    "data:propulsion:he_power_train:propeller:propeller_1:blade_twist", val=25.0, units="deg"
)
problem.set_val(
    "data:propulsion:he_power_train:propeller:propeller_1:diameter", val=50.0, units="inch"
)

problem.set_solver_print(level=0)

problem.run_driver()

print(problem.get_val("data:propulsion:he_power_train:propeller:propeller_1:solidity"))
print(problem.get_val("data:propulsion:he_power_train:propeller:propeller_1:activity_factor"))
print(
    problem.get_val("data:propulsion:he_power_train:propeller:propeller_1:blade_twist", units="deg")
)
print(
    problem.get_val("data:propulsion:he_power_train:propeller:propeller_1:diameter", units="inch")
)
print(problem.get_val("efficiency"))